# Bag of Words Text Classifier

The code below implements a simple bag of words text classifier.
- We tokenize the text, create a vocabulary and encode each piece of text in the dataset
- The lookup allows for extracting embeddings for each tokenized inputs
- The embedding vectors are added together with a bias vector
- The resulting vector is referred to as the scores
- The score are applied a softmax to generate probabilities which are used for the classification task

The code used in this notebook was inspired by code from the [official repo](https://github.com/neubig/nn4nlp-code) used in the [CMU Neural Networks for NLP class](http://www.phontron.com/class/nn4nlp2021/schedule.html) by [Graham Neubig](http://www.phontron.com/index.php). 

![img txt](../img/bow.png?raw=true)

In [4]:
import torch
import random
import torch.nn as nn

### Download the Data

In [2]:
%%capture

# download the files
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/dev.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/test.txt
!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/classes/train.txt

# create the data folders
!mkdir data data/classes
!cp dev.txt data/classes
!cp test.txt data/classes
!cp train.txt data/classes

### Read the Data

In [5]:
# 定义一个函数 read_data，用于从文件中读取数据并返回一个列表
def read_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.lower().strip()# 将每行转换为小写并去除首尾的空白字符
            line = line.split(' ||| ') # 切分每行数据，使用 ' ||| ' 作为分隔符
            data.append(line) # 将切分后的数据添加到列表中
    return data
# 调用 read_data 函数读取训练集和测试集数据
train_data = read_data('data/classes/train.txt')
test_data = read_data('data/classes/test.txt')

### Construct the Vocab and Datasets

In [6]:
# creating the word and tag indices
word_to_index = {} # 创建空字典，用于将单词映射为索引
word_to_index["<unk>"] = len(word_to_index) # 向字典中添加特殊符号 "<unk>" 并分配一个索引
tag_to_index = {} # 创建空字典，用于将标签映射为索引

# 定义一个函数 create_dict，用于将数据中的单词添加到字典 word_to_index 中
def create_dict(data, check_unk=False):
    for line in data:
        for word in line[1].split(" "): # 遍历每行数据中的单词
            if check_unk == False:
                if word not in word_to_index:# 如果单词不在字典中
                    word_to_index[word] = len(word_to_index) # 将单词添加到字典中并分配一个索引
            else:
                # 。如果 check_unk 为 True，并且单词不在字典中，则使用 <unk> 的索引进行代替。同时，还将每行数据的标签添加到 tag_to_index 字典中，并为每个独特的标签分配一个索引值。
                if word not in word_to_index:
                    word_to_index[word] = word_to_index["<unk>"]

        if line[0] not in tag_to_index:
            tag_to_index[line[0]] = len(tag_to_index)

# 调用 create_dict 函数处理训练集数据
create_dict(train_data)
# 调用 create_dict 函数处理测试集数据，同时将 check_unk 参数设置为 True
# 当 check_unk 为 True 时，如果单词不在字典中，将使用 "<unk>" 的索引值进行代替
create_dict(test_data, check_unk=True)

# create word and tag tensors from data
def create_tensor(data):
    for line in data:
        yield([word_to_index[word] for word in line[1].split(" ")], tag_to_index[line[0]])
# 将数据集中的单词和标签转换为索引表示的张量
# create_tensor 函数接受一个数据集 data 作为输入
# 使用 yield 关键字创建一个生成器
# 生成器返回每行数据的单词索引列表和对应的标签索引
# [word_to_index[word] for word in line[1].split(" ")] 将每行数据中的单词转换为索引表示
# tag_to_index[line[0]] 将每行数据的标签转换为索引表示
# 生成器将迭代遍历数据集的每一行，并返回对应的索引

# 将生成器的结果转换为列表表示，并分别赋值给 train_data 和 test_data
train_data = list(create_tensor(train_data))
test_data = list(create_tensor(test_data))
# 计算字典大小，用于嵌入层和分类器的维度
number_of_words = len(word_to_index)
number_of_tags = len(tag_to_index)

### Model

In [7]:
# cpu or gpu
device = "cuda" if torch.cuda.is_available() else "cpu"

# 定义一个继承自 torch.nn.Module 的 BoW 类（Bag-of-Words）
# BoW 类用于创建一个简单的神经网络模型
class BoW(torch.nn.Module):
    def __init__(self, nwords, ntags):
        super(BoW, self).__init__()
        self.embedding = nn.Embedding(nwords, ntags)# 创建一个嵌入层
        nn.init.xavier_uniform_(self.embedding.weight) # 初始化嵌入层的权重
        # 检查是否有可用的 GPU，选择相应的 Tensor 类型
        # 根据 ntags 创建一个偏置向量，requires_grad 设置为 True，表示梯度将会被计算
        # 这个偏置向量用于进行分类
        type = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
        self.bias = torch.zeros(ntags, requires_grad=True).type(type)

    def forward(self, x):
        emb = self.embedding(x) # 嵌入层的输出，维度为 seq_len x ntags (每个序列)
        out = torch.sum(emb, dim=0) + self.bias # 对嵌入层的输出进行求和，并加上偏置向量
        out = out.view(1, -1) # 重塑张量形状为 (1, ntags)
        return out

### Pretest the Model

In [8]:
# function to convert sentence into tensor using word_to_index dictionary
def sentence_to_tensor(sentence):
    return torch.LongTensor([word_to_index[word] for word in sentence.split(" ")])
# 定义一个函数 sentence_to_tensor，用于将句子转换为张量表示
# sentence_to_tensor 函数接受一个句子作为输入
# 使用列表推导式将句子中的每个单词转换为对应的索引，并使用 torch.LongTensor 转换为张量
# 返回句子的索引张量表示

# 根据是否有可用的 GPU，选择相应的 Tensor 类型
type = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor

# 调用 sentence_to_tensor 函数将句子 "i love dogs" 转换为索引张量，并将类型转换为先前定义的 Tensor 类型
out = sentence_to_tensor("i love dogs").type(type)

# 创建一个 BoW 类的实例，并将其移动到指定的 device 上
test_model = BoW(number_of_words, number_of_tags).to(device)

# 将输入张量 out 传递给 test_model 进行前向传播，得到模型的输出
test_model(out)

tensor([[ 0.0124,  0.0164, -0.0182, -0.0014, -0.0120]], device='cuda:0',
       grad_fn=<ViewBackward0>)

### Train the Model

In [9]:
# 创建一个 BoW 模型实例，参数为词汇表的大小和标签的数量
# 将模型移动到指定的设备(device)
model = BoW(number_of_words, number_of_tags).to(device)
# 定义一个交叉熵损失函数

criterion = nn.CrossEntropyLoss()
# 定义一个 Adam 优化器，用于更新模型参数
optimizer = torch.optim.Adam(model.parameters())
# 定义一个 Tensor 类型为 torch.LongTensor
type = torch.LongTensor

if torch.cuda.is_available():
    model.to(device)
    # 若可用 GPU，则将模型移动到指定的设备(device)，并将 Tensor 类型设置为 torch.cuda.LongTensor
    type = torch.cuda.LongTensor

# perform training of the Bow model
def train_bow(model, optimizer, criterion, train_data):
    # 进行训练
    for ITER in range(10):
        # 设置模型为训练模式
        model.train()
        # 随机打乱训练数据
        random.shuffle(train_data)
        # 初始化训练损失和正确分类的计数变量
        total_loss = 0.0
        train_correct = 0
        for sentence, tag in train_data:
            # 将句子转换为张量，并设置数据类型
            sentence = torch.tensor(sentence).type(type)
            # 将标签转换为张量，并设置数据类型
            tag = torch.tensor([tag]).type(type)
            # 通过模型进行前向传播，得到输出
            output = model(sentence)
            # 预测标签为输出的最大值所在的索引
            predicted = torch.argmax(output.data.detach()).item()
             # 计算损失
            loss = criterion(output, tag)
             # 累加损失值
            total_loss += loss.item()
            # 清空梯度
            optimizer.zero_grad()
            # 反向传播，计算梯度
            loss.backward()
            # 更新模型参数
            optimizer.step()

            if predicted == tag: train_correct+=1 # 统计训练集中分类正确的数量

        # perform testing of the model
        model.eval() # 设置模型为评估模式
        test_correct = 0 # 初始化测试集分类正确的计数变量
        for sentence, tag in test_data:
            # 将句子转换为张量，并设置数据类型
            sentence = torch.tensor(sentence).type(type)
            # 通过模型进行前向传播，得到输出
            output = model(sentence)
            # 预测标签为输出的最大值所在的索引
            predicted = torch.argmax(output.data.detach()).item()
            if predicted == tag: test_correct += 1 # 统计测试集中分类正确的数量
        
        # print model performance results
        log = f'ITER: {ITER+1} | ' \
            f'train loss/sent: {total_loss/len(train_data):.4f} | ' \
            f'train accuracy: {train_correct/len(train_data):.4f} | ' \
            f'test accuracy: {test_correct/len(test_data):.4f}'
        # 构建输出日志
        print(log)

# 调用 train_bow 函数进行模型训练
train_bow(model, optimizer, criterion, train_data)

ITER: 1 | train loss/sent: 1.4733 | train accuracy: 0.3631 | test accuracy: 0.4009
ITER: 2 | train loss/sent: 1.1216 | train accuracy: 0.6040 | test accuracy: 0.4118
ITER: 3 | train loss/sent: 0.9123 | train accuracy: 0.7117 | test accuracy: 0.4154
ITER: 4 | train loss/sent: 0.7688 | train accuracy: 0.7664 | test accuracy: 0.4140
ITER: 5 | train loss/sent: 0.6631 | train accuracy: 0.8065 | test accuracy: 0.4068
ITER: 6 | train loss/sent: 0.5814 | train accuracy: 0.8324 | test accuracy: 0.4059
ITER: 7 | train loss/sent: 0.5171 | train accuracy: 0.8507 | test accuracy: 0.4077
ITER: 8 | train loss/sent: 0.4640 | train accuracy: 0.8695 | test accuracy: 0.4036
ITER: 9 | train loss/sent: 0.4191 | train accuracy: 0.8830 | test accuracy: 0.3991
ITER: 10 | train loss/sent: 0.3818 | train accuracy: 0.8929 | test accuracy: 0.3964
